<a href="https://colab.research.google.com/github/Prajna1999/agent-langchain/blob/main/agent_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai
!pip install pinecone-client
!pip install pandas
!pip install typing
!pip install tqdm
!pip install langchain
!pip install wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26305 sha256=e23239c613a735449b34679dddb58f3a9d9ed3159631ff234d0a6f3d7df9f000
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=e0b85aff5ce204abaa5628cc0862112ff368c715b5522119d089707fbe56752d
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
import datetime
import json
import openai
import os
import pandas as pd
import pinecone
import re
from typing import List, Union
import zipfile


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser

In [5]:
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain

In [6]:
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage


In [7]:
#import LLM Wrapper
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI

In [8]:
#import coversational memory buffer
from langchain.memory import ConversationBufferWindowMemory

In [9]:
#Embeddings and vector stores
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

In [10]:
#vector store index
index_name='podcasts'

In [11]:
from getpass import getpass
PINECONE_API_KEY=getpass("Pinecone API key:")
PINECONE_ENV=getpass("Pinecone ENV:")

Pinecone API key:··········
Pinecone ENV:··········


In [12]:
#initialize a pinecone instance
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)



In [13]:
pinecone.whoami()

WhoAmIResponse(username='873894b', user_label='default', projectname='6e88b48')

In [14]:
if index_name in pinecone.list_indexes():
  pinecone.delete_index(index_name)

pinecone.create_index(name=index_name, dimension=1536)

In [15]:
pinecone.list_indexes()

['podcasts']

LLM agent


In [16]:
from tqdm.auto import tqdm


In [17]:
from getpass import getpass
SERPAPI_API_KEY=getpass('Enter Serp API key:')

Enter Serp API key:··········


In [18]:
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=1b2526de95a609f9b49ca81ea4ffef9a387f7a7c6133392201513563911d8bf8
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [1]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=bafa1cd44f914856aed542b1d6ed2d9532e1fa782cb538cb5b074e807f67482c
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [19]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

In [20]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [21]:
#initiate  a Search tool
# search=SerpAPIWrapper(serpapi_api_key="9f481e8bd9802dbb7c3673bcbb3a9f4fe46e8fed41649dde11e59b68a8b291ed")

#define a tool
tools=[
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="userful for when you need to answer questions about historical events"
    )
]

In [22]:
# Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
template = """Answer the following questions as best you can, but speaking as an erudite museum curator might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as an erudite museum curator when giving your final answer. DO NOT ANSWER IF THE QUESTION IS NOT ABOUT MUSEUMS OR HISTORY.

Question: {input}
{agent_scratchpad}"""

In [23]:
# set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
  template: str
  tools: List[Tool]

  def format_messages(self, **kwargs)->str:
    #format the messages in a particular way
    intermediate_steps=kwargs.pop("intermediate_steps")
    thoughts=""
    for action, observation in intermediate_steps:
      thoughts+=action.log
      thoughts+=f"\nObservation: {observation}\nThought: "

    # Set the agent_scratchpad variable to that variable to that value
    kwargs["agent_scratchpad"]=thoughts
    # create a tools variable from the list of tools provided
    kwargs["tools"]="\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])

    # create a list of tool names for the tools provided
    kwargs["tool_names"]=", ".join([tool.name for tool in self.tools])
    formatted=self.template.format(**kwargs)
    return [HumanMessage(content=formatted)]

prompt=CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]

)

In [24]:

#Custom output parser
class CustomOutputParser(AgentOutputParser):
  def parse(self, llm_output: str)-> Union[AgentAction, AgentFinish]:
    # check is agent should finish
    if  "final answer" in llm_output:
      return AgentFinish(
          # return values is generally always a dictionary
          # with single `output` key
          return_values={"output":llm_output.split("Final Answer:")[-1].strip()},
          log=llm_output,
      )
    # Parse out the action and action input
    regex=r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
    match=re.search(regex, llm_output, re.DOTALL)

    # If it can't parse the output it raises an error
    if not match:
      # raise ValueError(f"Could not parse LLM Output: `{llm_output}`")
      raise ValueError(f"Ahoy mate. I go all around the skynet to fetch you answers man.")

    action=match.group(1).strip()
    action_input=match.group(2)

    # return the action and action input
    return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser=CustomOutputParser()


In [25]:
#initiate LLM
from getpass import getpass
openai_api_key=getpass('Enter your OpenAI API key:')

Enter your OpenAI API key:··········


In [26]:
llm = ChatOpenAI(openai_api_key=openai_api_key,temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Using tools, the LLM chain and output_parser to make an agent
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    # We use "Observation" as our stop sequence so it will stop when it receives Tool output
    # If you change your prompt template you'll need to adjust this as well
    stop=["\nObservation:"],
    allowed_tools=tool_names,
    handle_parsing_errors=True,
)

In [27]:
# initiate the agent that would respond to our queries
agent_executor=AgentExecutor.from_agent_and_tools(agent=agent, tools=tools,verbose=True)

In [30]:
agent_executor.run("How may art pieces related to Egypt are there in British Museum,")



> Entering new AgentExecutor chain...
Thought: I need to find out the number of art pieces in the British Museum.
Action: Wikipedia
Action Input: "British Museum"

Observation:Page: British Museum
Summary: The British Museum is a public museum dedicated to human history, art and culture located in the Bloomsbury area of London. Its permanent collection of eight million works is the largest in the world. It documents the story of human culture from its beginnings to the present. The British Museum was the first public national museum to cover all fields of knowledge.In 2022 the museum received 4,097,253 visitors, an increase of 209 per cent from 2021. It ranked third in the list of most-visited art museums in the world.The museum was established in 1753, largely based on the collections of the Anglo-Irish physician and scientist Sir Hans Sloane. It first opened to the public in 1759, in Montagu House, on the site of the current building. The museum's expansion over the following 250 y

'The British Museum has a permanent collection of eight million works, making it the largest in the world.'

In [ ]:
# Next Step Text to SQL Queries. I need a proper wikipedia database for that where prominent Museum's databases are present. I need to build one for less prominent databases
